In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
import json
# Added 'census' and 'us' to the dependency list to utilize the census wrapper
from census import Census
from us import states

# Import API key
from config import api_key

c = Census(api_key)

In [2]:
file = "UNCLEANED_voter_data.csv"
voter_data = pd.read_csv(file)
voter_data.insert(3, 'Abbrv', voter_data["State Abv"])
voter_data = voter_data.iloc[:,:12]
voter_data = voter_data.dropna()
voter_data.to_csv("voter_data.csv",encoding="utf-8",index=False)

In [3]:
# Naming the data points for easier reading within the loop/API request
median_income = 'B19013_001E'
employed = 'B23025_004E'
unemployed = 'B23025_005E'
poverty = 'B17001_002E'
poverty_family = 'B17012_002E'
age = 'B01002_001E'
population = 'B01003_001E'
pop_white_alone = 'B02001_002E'
pop_black_alone = 'B02001_003E'
pop_american_indian_alone = 'B02001_005E'
pop_native_hawaiian_alone = 'B02001_006E'
pop_two_or_more_races = 'B02001_007E'
pop_hispanic_origin = 'B02001_008E'
median_home_value = 'B25077_001E'
median_gross_rent = 'B25064_001E'
commute_time_pub_transit = 'B08136_007E'
commute_time_solo_auto = 'B08136_003E'
commute_time_walked = 'B08136_011E'
transit_solo_auto = 'B08301_003E'
transit_pub_transit = 'B08301_010E'
transit_walked = 'B08301_019E'
transit_other = 'B08101_041E'
ed_none = 'B15003_002E'

In [4]:
# Starting the for loop to pull in the data fields. Having some trouble naming the four dataframes using the 'year' iterable...
# Any thoughts on how to do that?
# Run the for loop and it will give you the last dataframe in the series which is '2018', currently. I've formatted the df.
# This is almost certainly more information than we need, but I wanted to include data fields that might be relevent.
# We can always shrink down the dataframes to the fields we want to focus on.
years = [2012,2014,2016]
joint_df = pd.DataFrame()
for year in years:
    med_income = c.acs1.state(('NAME',
                              median_income,
                              employed,
                              unemployed,
                              poverty,
                              poverty_family,
                              age,
                              population,
                              pop_white_alone,
                              pop_black_alone,
                              pop_american_indian_alone,
                              pop_native_hawaiian_alone,
                              pop_two_or_more_races,
                              pop_hispanic_origin,
                              median_home_value,
                              median_gross_rent,
                              commute_time_solo_auto,
                              commute_time_pub_transit,
                              commute_time_walked,
                              transit_solo_auto,
                              transit_pub_transit,
                               # 'year=year' is where the 'year in years' is iterated. You can replace this with whatever
                               # year you want, take it out of for loop and run.
                              transit_walked),Census.ALL,year=year)
    census_df = pd.DataFrame(med_income)
    census_df = census_df.rename(columns={
        'NAME':'State',
        'B19013_001E':'Median Income',
        'B23025_004E':'Number Employed',
        'B23025_005E':'Number Unemployed',
        'B17001_002E':'Number Poverty',
        'B17012_002E':'Number Families in Poverty',
        'B01002_001E':'Median Age',
        'B01003_001E':'Population',
        'B02001_002E':'Pop: White Only',
        'B02001_003E':'Pop: Black Only',
        'B02001_005E':'Pop: American Indian Only',
        'B02001_006E':'Pop: Native Hawaiian Only',
        'B02001_007E':'Pop: Two or More Races',
        'B02001_008E':'Pop: Hispanic Origin',
        'B25077_001E':'Median Home Value',
        'B25064_001E':'Median Gross Rent',
        'B08136_007E':'Commute Time: Solo Auto',
        'B08136_003E':'Commute Time: Public Transit',
        'B08136_011E':'Commute Time: Walking',
        'B08301_003E':'Transit: Solo Auto',
        'B08301_010E':'Transit: Public Transit',
        'B08301_019E':'Transit: Walking'
    })
    census_df['Median Income'] = census_df['Median Income'].astype(float).map("${:,.2f}".format)
    census_df['Median Home Value'] = census_df['Median Home Value'].astype(float).map("${:,.2f}".format)
    census_df['Median Gross Rent'] = census_df['Median Gross Rent'].astype(float).map("${:,.2f}".format)
    census_df['Umemployment Rate'] = (census_df['Number Unemployed']/(census_df['Number Employed']+census_df['Number Unemployed'])).astype(float).map("{:.2%}".format)
    census_df['Poverty Rate'] = (census_df['Number Poverty']/census_df['Population']).astype(float).map("{:.2%}".format)
    census_df['Commute Time: Public Transit'] = (census_df['Commute Time: Public Transit']/census_df['Population'])
    census_df['Commute Time: Solo Auto'] = (census_df['Commute Time: Solo Auto']/census_df['Population'])
    census_df['Commute Time: Walking'] = (census_df['Commute Time: Walking']/census_df['Population'])
    census_df['Transit: Solo Auto'] = (census_df['Transit: Solo Auto']/census_df['Population']).astype(float).map("{:.2%}".format)
    census_df['Transit: Public Transit'] = (census_df['Transit: Public Transit']/census_df['Population']).astype(float).map("{:.2%}".format)
    census_df['Transit: Walking'] = (census_df['Transit: Walking']/census_df['Population']).astype(float).map("{:.2%}".format)
    census_df['Number Employed'] = census_df['Number Employed'].astype(int)
    census_df['Number Unemployed'] = census_df['Number Unemployed'].astype(int)
    census_df['Number Poverty'] = census_df['Number Poverty'].astype(int)
    census_df['Number Families in Poverty'] = census_df['Number Families in Poverty'].astype(int)
    census_df['Population'] = census_df['Population'].astype(int)
    census_df['Pop: White Only'] = census_df['Pop: White Only'].astype(int)
    census_df['Pop: Black Only'] = census_df['Pop: Black Only'].astype(int)
    census_df['Pop: American Indian Only'] = census_df['Pop: American Indian Only'].astype(int)
    census_df['Pop: Native Hawaiian Only'] = census_df['Pop: Native Hawaiian Only'].astype(int)
    census_df['Pop: Two or More Races'] = census_df['Pop: Two or More Races'].astype(int)
    census_df['Pop: Hispanic Origin'] = census_df['Pop: Hispanic Origin'].astype(int)
    census_df['Pop Rate: White Only'] = (census_df['Pop: White Only']/census_df['Population']).astype(float).map("{:.2%}".format)
    census_df['Pop Rate: Black Only'] = (census_df['Pop: Black Only']/census_df['Population']).astype(float).map("{:.2%}".format)
    census_df['Pop Rate: American Indian Only'] = (census_df['Pop: American Indian Only']/census_df['Population']).astype(float).map("{:.2%}".format)
    census_df['Pop Rate: Native Hawaiian Only'] = (census_df['Pop: Native Hawaiian Only']/census_df['Population']).astype(float).map("{:.2%}".format)
    census_df['Pop Rate: Two or More Races'] = (census_df['Pop: Two or More Races']/census_df['Population']).astype(float).map("{:.2%}".format)
    census_df['Pop Rate: Hispanic Origin'] = (census_df['Pop: Hispanic Origin']/census_df['Population']).astype(float).map("{:.2%}".format)
    census_df.insert(0, 'Year', year)
    joint_df = pd.concat([census_df, joint_df])
joint_df.to_csv("census_data_12_to_16.csv",encoding="utf-8",index=False)
joint_df

,Year,State,Median Income,Number Employed,Number Unemployed,Number Poverty,Number Families in Poverty,Median Age,Population,Pop: White Only,...,Transit: Walking,state,Umemployment Rate,Poverty Rate,Pop Rate: White Only,Pop Rate: Black Only,Pop Rate: American Indian Only,Pop Rate: Native Hawaiian Only,Pop Rate: Two or More Races,Pop Rate: Hispanic Origin
0,2016,Florida,"$50,860.00",9219488,585900,2973512,509616,42.1,20612439,15574165,...,0.67%,12,5.98%,14.43%,75.56%,16.06%,2.71%,0.06%,2.77%,2.58%
1,2016,Georgia,"$53,559.00",4735947,300529,1603013,296261,36.5,10310371,6054861,...,0.71%,13,5.97%,15.55%,58.73%,31.57%,3.87%,0.06%,2.90%,2.51%
2,2016,Hawaii,"$74,511.00",686487,31796,129569,18826,38.9,1428557,357994,...,2.34%,15,4.43%,9.07%,25.06%,1.81%,38.04%,10.12%,0.86%,23.92%
3,2016,Idaho,"$51,807.00",762788,37503,237445,41274,36.1,1683140,1510119,...,1.17%,16,4.69%,14.11%,89.72%,0.68%,1.45%,0.14%,3.93%,2.50%
4,2016,Illinois,"$60,960.00",6231419,419099,1621508,293044,37.9,12801539,9123763,...,1.44%,17,6.30%,12.67%,71.27%,14.17%,5.35%,0.04%,6.43%,2.51%
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47,2012,Puerto Rico,"$19,429.00",1094779,229733,1632533,363735,37.8,3667084,2536665,...,0.90%,72,17.34%,44.52%,69.17%,8.30%,0.37%,0.00%,9.60%,12.20%
48,2012,Alabama,"$41,574.00",2001628,222017,892564,180873,38.2,4822023,3333371,...,0.49%,01,9.98%,18.51%,69.13%,26.47%,1.22%,0.03%,1.09%,1.57%
49,2012,Alaska,"$67,712.00",351496,29864,72400,12542,33.8,731449,486551,...,3.60%,02,7.83%,9.90%,66.52%,3.58%,5.85%,1.13%,0.93%,7.97%
50,2012,Arizona,"$47,826.00",2757652,300531,1194506,212634,36.6,6553255,5164199,...,0.84%,04,9.83%,18.23%,78.80%,4.37%,2.91%,0.18%,6.42%,2.89%


In [5]:
voter_data_12to16 = voter_data.loc[voter_data["Year"].isin(years),:]
joint_df = pd.merge(joint_df,voter_data_12to16,on=["State","Year"],how="inner")
joint_df.to_csv("joint_data_12_to_16.csv",encoding="utf-8",index=False)
joint_df

,Year,State,Median Income,Number Employed,Number Unemployed,Number Poverty,Number Families in Poverty,Median Age,Population,Pop: White Only,...,ICPSR State Code,Alphanumeric State Code,Abbrv,VEP Total Ballots Counted,VEP Highest Office,VAP Highest Office,Total Ballots Counted (Estimate),Highest Office,Voting-Eligible Population (VEP),Voting-Age Population (VAP)
0,2016,Florida,"$50,860.00",9219488,585900,2973512,509616,42.1,20612439,15574165,...,43,10,FL,0.656,0.645,0.569,9580489.0,9420039.0,14601066,16565588
1,2016,Georgia,"$53,559.00",4735947,300529,1603013,296261,36.5,10310371,6054861,...,44,11,GA,0.598,0.591,0.526,4165405.0,4114732.0,6959963,7828207
2,2016,Hawaii,"$74,511.00",686487,31796,129569,18826,38.9,1428557,357994,...,82,12,HI,0.432,0.423,0.383,437664.0,428937.0,1012860,1120465
3,2016,Idaho,"$51,807.00",762788,37503,237445,41274,36.1,1683140,1510119,...,63,13,ID,0.609,0.592,0.550,710545.0,690255.0,1166706,1253925
4,2016,Illinois,"$60,960.00",6231419,419099,1621508,293044,37.9,12801539,9123763,...,21,14,IL,0.631,0.622,0.567,5666118.0,5589631.0,8985443,9866799
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129,2012,Washington,"$57,573.00",3204090,306984,915278,156411,37.5,6897012,5402912,...,73,48,WA,0.658,0.648,0.586,3172939.0,3125516.0,4822060,5329782
130,2012,Wyoming,"$54,901.00",288852,17113,71019,12425,36.9,576412,523798,...,68,51,WY,0.590,0.586,0.564,250701.0,249061.0,425142,441726
131,2012,Alaska,"$67,712.00",351496,29864,72400,12542,33.8,731449,486551,...,81,2,AK,0.589,0.587,0.553,301694.0,300495.0,511792,543763
132,2012,Arizona,"$47,826.00",2757652,300531,1194506,212634,36.6,6553255,5164199,...,61,3,AZ,0.530,0.526,0.465,2323579.0,2306559.0,4387900,4959270
